In [ ]:
# imports
import pandas as pd
import numpy as np
import random, time
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.collections import LineCollection
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm, trange
import json

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Embeddings/Text/embedding_roberta_train_latest.csv')
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Embeddings/Text/embedding_roberta_dev_unseen_latest.csv')
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Embeddings/Text/embedding_roberta_test_seen_latest.csv')
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Embeddings/Text/embedding_roberta_test_unseen_latest.csv')


#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Roberta_retrain/retrain_embedding_1_train.csv')
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Roberta_retrain/retrain_embedding_1_val.csv')
#text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Roberta_retrain/retrain_embedding_1_test_seen.csv')
text_embeddings_train = pd.read_csv('/content/drive/MyDrive/522-project/Roberta_retrain/retrain_embedding_1_test_unseen.csv')

In [ ]:
text_embeddings_train

,Unnamed: 0,0,1
0,0,15740,"[-0.02474036253988743, 0.054800838232040405, 0..."
1,1,38794,"[0.005572420544922352, 0.033805347979068756, -..."
2,2,60792,"[-0.00891077984124422, 0.004879706539213657, 0..."
3,3,71824,"[-0.017494771629571915, 0.05120307207107544, 0..."
4,4,4796,"[0.022563030943274498, 0.04587974771857262, 0...."
...,...,...,...
1995,1995,71352,"[-0.014976058155298233, 0.048641420900821686, ..."
1996,1996,2164,"[-0.006845325697213411, 0.02922895736992359, 0..."
1997,1997,3587,"[-0.012692566961050034, 0.05309787020087242, 0..."
1998,1998,47839,"[-0.004035735968500376, 0.013722299598157406, ..."


In [ ]:
import json

text_embeddings_train["1"]= text_embeddings_train["1"].apply(lambda x : json.loads(x))
# text_embeddings_train ["1"]= json.loads(text_embeddings_train ["1"].apply(lambda: x : json))

In [ ]:
split_df = pd.DataFrame(text_embeddings_train['1'].tolist())

In [ ]:
split_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.024740,0.054801,0.071823,-0.025032,0.071833,0.058036,0.055040,0.005771,0.055570,0.036501,...,0.015269,0.072266,-0.047980,0.005068,0.076145,-0.007374,-0.012966,-0.087614,0.040363,-0.017843
1,0.005572,0.033805,-0.015727,-0.045046,0.056712,0.003052,0.032610,-0.003532,0.050244,0.048040,...,0.058084,0.006788,-0.025066,0.037624,0.013589,-0.023877,-0.004047,-0.014419,0.069658,-0.103777
2,-0.008911,0.004880,0.047518,-0.018775,0.070224,-0.026994,0.005044,0.041162,0.055882,0.020442,...,0.022596,0.036880,-0.026062,0.062280,-0.037752,0.013773,-0.012445,-0.037078,0.065592,-0.066062
3,-0.017495,0.051203,0.007668,-0.066592,0.102139,-0.020125,0.009824,0.042612,0.035486,0.030007,...,0.022162,0.003353,-0.014462,0.062976,-0.025840,0.005613,0.026096,0.014619,0.041803,-0.141341
4,0.022563,0.045880,0.031437,-0.068524,0.067283,-0.002515,0.023688,-0.014744,0.075807,0.031510,...,0.048119,0.032094,0.036675,0.048916,0.025368,-0.038169,-0.037226,-0.014845,0.065336,-0.061386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.014976,0.048641,0.006241,-0.052580,0.048231,0.004684,0.005850,0.023471,0.042598,0.025180,...,0.031405,0.010093,0.014699,0.067035,0.021178,0.023952,-0.045772,0.019612,0.054043,-0.084120
1996,-0.006845,0.029229,0.008552,-0.049752,0.062002,0.019634,0.028001,0.008200,0.024754,0.041944,...,0.024341,0.033089,-0.001250,0.062732,0.002471,0.001162,-0.034572,-0.045419,0.032393,-0.103905
1997,-0.012693,0.053098,0.065099,0.001913,0.100865,0.017048,0.035771,0.032162,0.005809,0.069348,...,0.039715,-0.017721,0.034004,0.052581,0.044101,-0.001249,-0.001062,-0.049014,0.051102,-0.077510
1998,-0.004036,0.013722,0.032263,-0.043923,0.055121,-0.009848,0.011665,0.008052,0.004991,0.024087,...,0.021033,-0.032148,-0.023820,0.043022,0.024345,-0.013305,-0.029044,-0.029892,0.037183,-0.059996


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=150).fit(split_df)
transform_data = pd.DataFrame(pca.transform(split_df))
# principalDf = []
# for i in text_embeddings_train['1']:
#   principalComponents = pca.fit_transform(np.array(i).reshape(-1,1))
#   principalDf.append(pd.DataFrame(data = principalComponents))

In [ ]:
pca.explained_variance_

array([7.64183239e+00, 7.59048525e-01, 7.41266740e-02, 4.29355552e-02,
       3.77924769e-02, 3.33857306e-02, 2.37311460e-02, 1.63977480e-02,
       1.27037464e-02, 1.16896655e-02, 9.77688425e-03, 9.22906546e-03,
       7.48897575e-03, 7.37887319e-03, 6.70039301e-03, 6.37087065e-03,
       5.44806687e-03, 5.28342559e-03, 5.03908551e-03, 4.95758886e-03,
       4.79566438e-03, 4.47309903e-03, 4.37737067e-03, 4.19750475e-03,
       4.01302535e-03, 3.87046544e-03, 3.79063365e-03, 3.65072677e-03,
       3.50042621e-03, 3.38662574e-03, 3.29328294e-03, 3.18363639e-03,
       3.09192208e-03, 3.02406002e-03, 2.90536702e-03, 2.87600869e-03,
       2.79306341e-03, 2.77046453e-03, 2.72069414e-03, 2.67548353e-03,
       2.55355341e-03, 2.50158667e-03, 2.45429427e-03, 2.42322135e-03,
       2.36271766e-03, 2.31070676e-03, 2.29177910e-03, 2.24570913e-03,
       2.21022341e-03, 2.15466350e-03, 2.12685160e-03, 2.06452063e-03,
       2.02100647e-03, 1.99100556e-03, 1.97805310e-03, 1.96381146e-03,
      

In [ ]:
transform_data
transform_data['combine'] = transform_data.values.tolist()

In [ ]:
# transform_data["id"] = text_embeddings_train["0"]

In [ ]:
# transform_data.drop(columns="id", inplace=True)

In [ ]:
transform_data_combined = pd.concat([text_embeddings_train["0"],transform_data['combine']], axis =1)

In [ ]:
transform_data_combined

,0,combine
0,15740,"[-0.7551414358304682, -0.22236742778676674, 0...."
1,38794,"[-0.7276214785701451, -0.34057431130636034, -0..."
2,60792,"[-0.0593567224854942, -0.32344481669747777, 0...."
3,71824,"[-0.6763810138629107, -0.3651585244328396, -0...."
4,4796,"[-0.5086866409977331, -0.23745743677223244, 0...."
...,...,...
1995,71352,"[0.04879079395939581, -0.4073956004339999, -0...."
1996,2164,"[-0.21833574005772385, -0.38754094976730014, -..."
1997,3587,"[-0.06359462380121628, -0.12964778777832633, 0..."
1998,47839,"[-5.216263943151493, 1.1053855103699217, 0.106..."


In [ ]:
#transform_data_combined.to_csv('/content/drive/MyDrive/522-project/Embeddings/Roberta_retrain150/retrain_embedding_1_test_seen.csv')
transform_data_combined.to_csv('/content/drive/MyDrive/522-project/Embeddings/Roberta_retrain150/retrain_embedding_1_test_unseen.csv')
#transform_data_combined.to_csv('/content/drive/MyDrive/522-project/Embeddings/Roberta_retrain150/retrain_embedding_1_train.csv')
#transform_data_combined.to_csv('/content/drive/MyDrive/522-project/Embeddings/Roberta_retrain150/retrain_embedding_1_val.csv')